# Task 1 Multiplier

## Problem statement

In this task, we wish to make a multiplier quantum circuit. For this, we design the input to be two positive integers to a function which will process a quantum algorithm that makes the multiplier (see Draper adder) and returns the result in an integer.

You cannot use any implementation already designed by the framework. If possible, consider printing out your quantum circuit.

### Example usage

A = multiplier(5,6)

print(A)

30

### Bonus

Use your proposal to design different inputs, and check the limitations of your simulator and framework, consider number of qubits, time of execution, the depth of the quantum circuit and number of the gates.


### References

* Addition on a Quantum Computer
https://arxiv.org/pdf/quant-ph/0008033.pdf 

* T-count Optimized Design of Quantum Integer Multiplication  
https://arxiv.org/pdf/1706.05113.pdf 

* Quantum arithmetic with the Quantum Fourier Transform 
https://arxiv.org/pdf/1411.5949.pdf

## Import statements

The following block is to contain all necessary import statements for this notebook. Run it before running any other code cell.

In [1]:
import tequila as tq
# import numpy as np
from numpy import binary_repr as binrep
from numpy import ceil, log2, pi
from typing import Union

## QFT Review

TODO: a review of what QFT does

Consider a system of $n$ qubits in the computational basis $\{\left| j \right\rangle: j\in [N]\}$ with $N=2^n$. Then, the QFT operation is a linear map such that, for any computational basis state $\left| x\right\rangle$, we have
$$QFT\left| x\right\rangle = \frac1{\sqrt{N}} \sum_{j\in [N]} e^{i\frac{2\pi x j}{N}} \left| j\right\rangle.$$
The QFT operation can be implemented explicitly in a quantum circuit using $\mathcal{O}(n^2)$ quantum gates (using only Hadamard, and controlled phase gates), as follows.

In [2]:
def qft(bits: Union[int, list[int]]) -> tq.QCircuit:
    """
    Returns a circuit implementing QFT on n qubits, where n = len(bits)
    
    The list of bits is ordered in decreasing priority of bits, i.e. with the MSB as bits[0]
    """
    circ = tq.QCircuit()
    if isinstance(bits, int):
        n, bits = bits, list(range(bits))
    else:
        n = len(bits)
    
    for i in range(n-1):
        circ += tq.gates.H(bits[i])
        for j in range(i+1, n):
            m = j - i + 1
            phi = 2 * pi / (2 ** m)
            circ += tq.gates.Phase(target=bits[i], control=bits[j], angle=phi)
    circ += tq.gates.H(bits[-1])
    
    return circ

In [3]:
def test_qft():
    tq.draw(qft(4))

test_qft()
print(qft(4))

                     ┌──┐   ┌────────┐             ┌──┐
0: ───H──────────S────T──────Z^(1/8)──────────────────────────────────
                 │    │      │
1: ───T──────────@────┼H─────┼───────────S──────────T─────────────────
                      │      │           │          │
2: ───Z^(1/8)─────────@──────┼──────T────@──────────┼H────────S───────
                             │                      │         │
3: ───Z^(1/16)───────────────@───────────Z^(1/8)────@─────T───@───H───
                     └──┘   └────────┘             └──┘
circuit: 
H(target=(0,))
Phase(target=(0,), control=(1,), parameter=1.5707963267948966)
Phase(target=(0,), control=(2,), parameter=0.7853981633974483)
Phase(target=(0,), control=(3,), parameter=0.39269908169872414)
H(target=(1,))
Phase(target=(1,), control=(2,), parameter=1.5707963267948966)
Phase(target=(1,), control=(3,), parameter=0.7853981633974483)
H(target=(2,))
Phase(target=(2,), control=(3,), parameter=1.5707963267948966)
H(target=(3,))



## QFT Adder

TODO: a review of implementing a Draper adder using QFT

In [4]:
def qft_ancilla(bits: Union[int, list[int]], anc: int) -> tq.QCircuit:
    """
    Returns a circuit implementing QFT on (n+1) qubits, using 1 ancillary qubit, viz. anc, where n = len(bits)
    
    Preconditions:
        - if isinstance(bits, list): anc not in bits
    """
    if isinstance(bits, int):
        n, bits = bits, list(range(bits))
    else:
        n = len(bits)
    
    bits[:0] = [anc]
    
    return qft(bits)


def adder(n1: int, n2: int) -> tuple[tq.QCircuit, int]:
    """
    Returns a circuit implementing a QFT adder to add two integers $n_1$ and $n_2$, and the sum $n_1 + n_2$
    """
    circ = tq.QCircuit()
    
    # initialize variables
    bin1, bin2 = binrep(n1), binrep(n2)
    num_bits1, num_bits2 = int(ceil(log2(n1))), int(ceil(log2(n2)))
    n_bits = max(num_bits1, num_bits2)
    bin1, bin2 = '0' * (n_bits - len(bin1)) + bin1, '0' * (n_bits - len(bin2)) + bin2
    
    # prepare n1 and n2 in comp basis (first n_bits qubits is n1, second n_bits qubits is n2)
    for i in range(n_bits):
        if bin1[i] == '1':
            circ += tq.gates.X(i)
        if bin2[i] == '1':
            circ += tq.gates.X(n_bits + i)
    
    # prepare QFT(n2 + 0)
    circ += qft_ancilla(list(range(n_bits, 2 * n_bits)), 2 * n_bits)
    
    # iterate through, and add controlled phase gates
    for i in range(n_bits):
        ...
    
    for i in range(2 * n_bits - 1, n_bits - 1, -1):
        for j in range(n_bits):
            ...
    
    return circ

In [5]:
def test_qft_ancilla():
    qft_ancilla(4, 90)


def test_adder():
    adder(17, 8)


test_qft_ancilla()
test_adder()

## QFT Multiplier


In [6]:
def multiplier(num1: int, num2: int) -> int:
    """
    Inputs:
        number_1 : integer positive value that is the first parameter to the multiplier function,
        number_2 : integer positive value that is the second parameter to the multiplier function.
    
    Preconditions:
        number_1 > 0
        number_2 > 0
    
    Output:
        the positive integer value of the multiplication between number_1 and number_2
     """

    # initialize variables
    bin1, bin2 = binrep(num1), binrep(num2)
    num_bits1, num_bits2 = int(ceil(log(num1))), int(ceil(log(num2)))
    
    circ = tq.QCircuit()    # initialize circuit
    
    return 0 # the result of the quantum circuit into an integer value

In [7]:
print(list(range(2, 2 * 2)))

[2, 3]
